# Get Data

In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import numpy as np 

key_path = f"../../secrets/google_creds.json"
credentials = service_account.Credentials.from_service_account_file(key_path,
                                                                    scopes=[
                                                                        "https://www.googleapis.com/auth/cloud-platform"], )
client = bigquery.Client(credentials=credentials, project=credentials.project_id)
query = """SELECT * FROM
`net_expr.trials` AS t
INNER JOIN `net_expr.person` as p
ON t.participant_id = p.participant_id
WHERE 
p.is_test is FALSE AND p.participant_id != 'seed' 
"""
query_job = client.query(query)
df = query_job.to_dataframe()

# Two cases where a test response (response_text == "Test") was not flagged --> drop, note in paper
er_ids = ['dbe96241-4d64-4ae6-bb6d-15e5e45b0760', 'e6fdd18f-de53-4159-965a-656aa75ba88d']
df = df.query("response_id not in @er_ids")

## Data Checks

In [2]:
def verify_rid_unique():
    print("TEST: Does every response id occur only once?")
    unique_rds = len(df['response_id'].unique())
    all_rids = len(df['response_id'].tolist())
    if all_rids == unique_rds:
        print("YES: Every response id only occurs once")
    else:
        print("NO: Some response ids occur twice")
    
    
def verify_no_response_blank():
    print("TEST: Is it the case that all response texts have some words")
    blanks = len([x for x in df['response_id'].tolist() if x is None])
    if blanks == 0:
        print("YES: All responses have some words")
    else:
        print("NO: Some responses don't have words")
    
        
verify_rid_unique()
print()
verify_no_response_blank()

TEST: Does every response id occur only once?
YES: Every response id only occurs once

TEST: Is it the case that all response texts have some words
YES: All responses have some words


# Clean Data

## Clean source column

We tracked where traffic came from by appending different things to the request arguments for the experiment site. E.g: If posting the experiment on Facebook we would make the URL `experiment.com?from=facebook` so then `request_args` would be equal to `from=facebook`. But in some cases, no `request args` are available. This happens if somehow the request_args are stripped. There should be very few cases of this. 

In [3]:
def verify_other_source():
    print("TEST: Does every source marked as OTHER have no request args?")
    other_request_args = df.query("source=='other'")['request_args'].tolist()
    len_other = len(other_request_args)
    len_other_no_args = len([x for x in other_request_args if x == 'None'])
    if len_other == len_other_no_args:
        print("YES: For every source marked as `other`, we marked it as `other` because it does not have request args")
    else:
        print("NO: We missed categorizing some sources")
    
    

def categorize_request_args(request_args):
    request_args = request_args.lower()
    if 'facebook' in request_args:
        return 'facebook'
    elif 'chatgpt' in request_args:
        return 'r/chatgpt'
    elif 'sample' in request_args:
        return 'r/samplesize'
    elif 'writing' in request_args:
        return 'r/writing'
    elif 'internet' in request_args:
        return 'r/InternetIsBeautiful'
    elif 'creative' in request_args:
        return 'Creative Mornings newsletter'
    elif 'poetry' in request_args:
        return 'r/poetry'
    elif 'results' in request_args or 'share' in request_args:
        return 'share'
    else:
        return 'other'

df['source'] = df['request_args'].apply(categorize_request_args)
verify_other_source()

df['source'].value_counts()

TEST: Does every source marked as OTHER have no request args?
YES: For every source marked as `other`, we marked it as `other` because it does not have request args


Creative Mornings newsletter    1439
r/InternetIsBeautiful           1131
r/samplesize                     390
share                            251
r/chatgpt                         79
r/writing                         30
other                             23
r/poetry                          15
facebook                           7
Name: source, dtype: int64

## Clean Age Columns
Participants entered their age, which was an int that had to be over 18. But let's make sure nobody entered anything weird -- and by weird we will look at responses over 70 years old. From inspecting the data, it appears that responses above 74 are troll responses -- relabel as NA and report in paper. 

In [4]:
dfp = df.drop_duplicates(subset=['participant_id'])
dfp['age'].describe()

count    656.000000
mean      36.859756
std       29.071985
min       18.000000
25%       27.000000
50%       34.000000
75%       40.000000
max      444.000000
Name: age, dtype: float64

In [5]:
over_seven = dfp[dfp['age']>=70]
over_seven

,response_id,participant_id,response_text,response_date,condition,condition_order,item,world,init_array,ranked_array,...,country,age,is_test_1,referer,request_args,is_prolific_1,prolific_id,gender,gender_describe,source
711,2e5141b6-814c-4e5f-89f6-9fe6155915e3,7f067979-5f4b-4729-a61f-827d82f929ff,Building a straw ball house,2023-07-14 03:00:50,f_u,0,tire,3,"[723fd22b-6121-4935-82a1-54dfcecec5d9, e7726b1...","[a01068a1-ea4a-4c84-b9df-082ed1fe1b54, fd61a41...",...,United States,74,False,None,from=news&sub=creative&mc_cid=5c0a1bc1ef&mc_ei...,False,None,prefer_self_describe,She/her/they,Creative Mornings newsletter
1859,5c366ee2-b272-454b-8f5f-b4b6e54e67b1,b2269ab4-bc4b-4049-b234-e240644e2cf7,to make into a bicycle tire made out of shoes,2023-07-08 17:26:18,h,0,shoe,3,"[human_seed39_world3, human_seed37_world3, hum...","[human_seed38_world3, human_seed40_world3, hum...",...,China,420,False,https://createwithai.herokuapp.com/results/1b7...,how=results,False,None,prefer_not_disclose,None,share
2013,d2d498ec-58a0-4286-bb03-e4aa50f0486a,4c7db1ec-9ae0-4b58-b4d3-8186d5599d90,Portable cell-phone,2023-07-08 15:21:19,f_l,0,shoe,1,"[43d81087-73e7-41ba-949d-0041d1cd8811, f8ecf73...","[43d81087-73e7-41ba-949d-0041d1cd8811, f8ecf73...",...,Austria,432,False,https://www.reddit.com/,from=reddit&sub=internet,False,None,man,None,r/InternetIsBeautiful
2171,4735ccd1-f9b8-4d39-a115-5ea58265e0a2,8459a1e8-f7a7-46b1-82a2-0d99056ac3f4,store krabby patty secret formula,2023-07-08 15:49:57,f_l,0,bottle,1,"[d2a4126d-b581-4cb9-ba80-60dfc39d227b, 36b21de...","[d6829192-dce2-4a9d-9d46-e836acd74201, 36b21de...",...,Brazil,444,False,https://www.reddit.com/,from=reddit&sub=internet,False,None,prefer_not_disclose,None,r/InternetIsBeautiful
2939,cde0fcff-bf2b-45b7-bcd5-a32386664c8f,ae94364d-2fb8-47d0-b909-1c9e0d97aeff,Cut legs off to use as shorts,2023-07-08 15:05:43,m_l,0,pants,0,"[ai_seed562, f28fb317-4ea6-418f-9040-dd180452c...","[ai_seed304, f28fb317-4ea6-418f-9040-dd180452c...",...,Canada,111,False,https://out.reddit.com/,from=reddit&sub=internet,False,None,man,None,r/InternetIsBeautiful


In [6]:
print(dfp[dfp['age']>74][['participant_id', 'age']].to_latex(caption="Users whose age we replaced with missing"))

\begin{table}
\centering
\caption{Users whose age we replaced with missing}
\begin{tabular}{llr}
\toprule
{} &                        participant\_id &  age \\
\midrule
1859 &  b2269ab4-bc4b-4049-b234-e240644e2cf7 &  420 \\
2013 &  4c7db1ec-9ae0-4b58-b4d3-8186d5599d90 &  432 \\
2171 &  8459a1e8-f7a7-46b1-82a2-0d99056ac3f4 &  444 \\
2939 &  ae94364d-2fb8-47d0-b909-1c9e0d97aeff &  111 \\
\bottomrule
\end{tabular}
\end{table}



In [7]:
def fix_age(x):
    if pd.isnull(x):  # Check for NaN values
        return x
    elif x <= 74:
        return int(x)
    else:
        return np.NaN

df['age'] = df['age'].apply(lambda x: fix_age(x))
df['age'] = df['age'].astype('Int64')

# Clean columns

## Drop unused columns

Note `is_troll` was not used; the relevant column is `is_profane`. 

In [8]:
df.columns

Index(['response_id', 'participant_id', 'response_text', 'response_date',
       'condition', 'condition_order', 'item', 'world', 'init_array',
       'ranked_array', 'is_test', 'duration', 'is_troll', 'is_profane',
       'is_prolific', 'participant_id_1', 'creativity_human', 'creativity_ai',
       'dt', 'ai_feeling', 'country', 'age', 'is_test_1', 'referer',
       'request_args', 'is_prolific_1', 'prolific_id', 'gender',
       'gender_describe', 'source'],
      dtype='object')

In [9]:
to_drop = ["is_prolific", "is_prolific_1", "prolific_id", "is_test_1", "is_troll", "dt", 'participant_id_1']
for x in to_drop:
    try:
        df = df.drop(columns = [x], axis=0)
    except KeyError:
        pass

## Fix column types and names

In [10]:
df['creativity_human'] = df['creativity_human'].astype('Int64')
df['creativity_ai'] = pd.to_numeric(df['creativity_ai'], errors='coerce').astype('Int64')
df = df.rename(columns={'world': 'response_chain'})

# Add trial_no

In [11]:
df['response_date'] = pd.to_datetime(df['response_date'])  # ensure response_date is datetime
df['trial_no'] = df.groupby(['response_chain', 'condition', 'item'])['response_date'].rank(method='first').astype(int)

## Save data

In [12]:
df.to_csv("../../data/experiment_data/data_clean.csv")

# Descriptive Stats

In [13]:
import seaborn as sns
import matplotlib.pyplot as plt
def make_aesthetic(hex_color_list=None):
    """Make Seaborn look clean"""
    sns.set(style='white', context='poster', font_scale=0.8)
    if not hex_color_list:
        hex_color_list = [
        "#826AED", # Medium slate blue
        "#00A896", # Persian green
        "#D41876", # Telemagenta
        "#89DAFF", # Pale azure
        "#F7B2AD", # Melon
        "#342E37", # Dark grayish-purple
        "#7DCD85", # Emerald
        "#E87461", # Medium-bright orange
        "#E3B505", # Saffron
        "#2C3531", # Dark charcoal gray with a green undertone
        "#D4B2D8", # Pink lavender
        "#7E6551", # Coyote
        "#F45B69", # Vibrant pinkish-red
        "#020887", # Phthalo Blue
        "#F18805"  # Tangerine
        ]
    sns.set_palette(sns.color_palette(hex_color_list))
    plt.rcParams['font.family'] = 'Arial'
    plt.rcParams['axes.spines.right'] = False
    plt.rcParams['axes.spines.top'] = False
    plt.rcParams['axes.titlelocation'] = 'left'
    plt.rcParams['axes.titleweight'] = 'bold'
    plt.rcParams['axes.grid'] = True
    plt.rcParams['grid.linestyle'] = '--'
    plt.rcParams['grid.alpha'] = 0.5
    plt.rcParams['axes.facecolor'] = 'white'
    plt.rcParams['legend.frameon'] = True
    plt.rcParams['legend.framealpha'] = 0.8
    plt.rcParams['legend.facecolor'] = 'white'
    plt.rcParams['savefig.transparent'] = True
    plt.rcParams['savefig.bbox'] = 'tight'
    plt.rcParams['savefig.pad_inches'] = 0.1
    plt.rcParams['figure.autolayout'] = True
make_aesthetic()

### Meta Info

In [14]:
df_new = pd.DataFrame(index=["Value"])
df_new['Unique Countries'] = [int(len(df['country'].unique()))]
df_new['Total Responses'] = [int(len(df))]
df_new['Unique Participants'] = [df['participant_id'].nunique()]
df_new['Avg Responses/Participant'] = df_new['Total Responses'] / df_new['Unique Participants']
df_new['Avg Duration/Response'] = [df['duration'].mean()]

# Round the averages to two decimal places
df_new['Avg Responses/Participant'] = df_new['Avg Responses/Participant'].round(2)
df_new['Avg Duration/Response'] = df_new['Avg Duration/Response'].round(2)

# Print as LaTeX table
latex_table = df_new.T.to_latex(index=True, caption="Summary Statistics of Experiment", label="overall_stats")
print(latex_table)


\begin{table}
\centering
\caption{Summary Statistics of Experiment}
\label{overall_stats}
\begin{tabular}{lr}
\toprule
{} &    Value \\
\midrule
Unique Countries          &    48.00 \\
Total Responses           &  3365.00 \\
Unique Participants       &   822.00 \\
Avg Responses/Participant &     4.09 \\
Avg Duration/Response     &   144.80 \\
\bottomrule
\end{tabular}
\end{table}



### Sources

In [15]:
participants = df.groupby('source')['participant_id'].nunique()
percentage_participants = (participants / df['participant_id'].nunique()) * 100
trials = df['source'].value_counts()
percentage_trials = (trials / df.shape[0]) * 100
result = pd.DataFrame({
    'Participants': participants,
    'Percentage of Participants': percentage_participants,
    'Trials': trials,
    'Percentage of Trials': percentage_trials,
})

result = result.sort_values('Participants', ascending=False)
result['Participants (N, % of total)'] = result['Participants'].astype(str) + ' (' + result['Percentage of Participants'].round(1).astype(str) + '%)'
result['Trials (N, % of total)'] = result['Trials'].astype(str) + ' (' + result['Percentage of Trials'].round(1).astype(str) + '%)'
result = result.drop(columns=['Participants', 'Percentage of Participants', 'Trials', 'Percentage of Trials'])
latex_table = result.to_latex(caption="Sources of participants and trials", label="sources")

print(latex_table)


\begin{table}
\centering
\caption{Sources of participants and trials}
\label{sources}
\begin{tabular}{lll}
\toprule
{} & Participants (N, \% of total) & Trials (N, \% of total) \\
\midrule
Creative Mornings newsletter &                  333 (40.5\%) &           1439 (42.8\%) \\
r/InternetIsBeautiful        &                  298 (36.3\%) &           1131 (33.6\%) \\
r/samplesize                 &                   93 (11.3\%) &            390 (11.6\%) \\
share                        &                    61 (7.4\%) &             251 (7.5\%) \\
r/chatgpt                    &                    19 (2.3\%) &              79 (2.3\%) \\
r/writing                    &                     7 (0.9\%) &              30 (0.9\%) \\
other                        &                     6 (0.7\%) &              23 (0.7\%) \\
r/poetry                     &                     3 (0.4\%) &              15 (0.4\%) \\
facebook                     &                     2 (0.2\%) &               7 (0.2\%) \\
\

In [16]:
import numpy as np
import pandas as pd

class DescriptiveStats:
    def __init__(self, df):
        self.df = df

    def generate_categorical_string(self, column, decimal_places):
        percentages = self.df[column].value_counts(normalize=True, dropna=True) * 100
        percentage_nan = self.df[column].isna().mean() * 100
        percentage_string = ", ".join([f"{percentage:.{decimal_places}f}\% {index}" for index, percentage in percentages.items()])
        percentage_string += f". {percentage_nan:.{decimal_places}f}% of responses were missing."
        return f"Of the non-missing values, the sample was {percentage_string}"

    def print_latex_table_categorical(self, column, caption=None, label=None, decimal_places=2):
        value_counts = self.df[column].value_counts(dropna=False)
        value_counts_percentages = self.df[column].value_counts(normalize=True, dropna=False) * 100
        df_output = pd.DataFrame({column: value_counts.index, 
                                  'Counts (% of total)': [f"{count} ({percentage:.{decimal_places}f}%)" for count, percentage in zip(value_counts.values, value_counts_percentages.values)]})
        df_output[column] = df_output[column].fillna("Missing")
        df_output.set_index(column, inplace=True)
        latex_table = df_output.to_latex(index=True, caption=caption or f"Distribution of {column.replace('_', ' ').title()}", label=label or f"dist_{column}")
        print(latex_table)
        print(self.generate_categorical_string(column, decimal_places))
    
    def generate_continuous_string(self, column, decimal_places):
        mean_value = round(self.df[column].mean(), decimal_places)
        std_dev = round(self.df[column].std(), decimal_places)
        stats_string = f"The mean {column} was {mean_value:.{decimal_places}f} (SD = {std_dev:.{decimal_places}f})."
        return stats_string

    def print_latex_table_continuous(self, columns, caption=None, label=None, decimal_places=2):
        if not isinstance(columns, list):
            columns = [columns]
        stats_list = []
        for column in columns:
            mean_value = round(self.df[column].mean(), decimal_places)
            median_value = round(self.df[column].median(), decimal_places)
            std_dev = round(self.df[column].std(), decimal_places)
            quartile_25 = round(np.percentile(self.df[column].dropna(), 25), decimal_places)
            quartile_75 = round(np.percentile(self.df[column].dropna(), 75), decimal_places)
            stats_series = pd.Series({'Mean': mean_value,
                                      'Median': median_value,
                                      'SD': std_dev,
                                      '25th Percentile': quartile_25,
                                      '75th Percentile': quartile_75},
                                     name=column)
            stats_list.append(stats_series)

        df_output = pd.concat(stats_list, axis=1)
        latex_table = df_output.T.to_latex(index=True, caption=caption or "Distribution of Continuous Variables", label=label)
        print(latex_table)
        for column in columns:
            print(self.generate_continuous_string(column, decimal_places))

ds = DescriptiveStats(df)
dp = DescriptiveStats(df.drop_duplicates(subset=['participant_id']))

dp.print_latex_table_continuous(['age', 'creativity_ai', 'creativity_human'], decimal_places=2, caption="Descriptive Stats")

dp.print_latex_table_categorical('gender', decimal_places=0)

dp.print_latex_table_categorical('ai_feeling', decimal_places=0)


\begin{table}
\centering
\caption{Descriptive Stats}
\begin{tabular}{lrrrrr}
\toprule
{} &   Mean &  Median &     SD &  25th Percentile &  75th Percentile \\
\midrule
age              &  34.93 &    33.5 &  10.87 &             27.0 &            40.00 \\
creativity\_ai    &  58.04 &    60.0 &  26.54 &             40.0 &            76.25 \\
creativity\_human &  58.86 &    62.0 &  23.51 &             45.0 &            75.00 \\
\bottomrule
\end{tabular}
\end{table}

The mean age was 34.93 (SD = 10.87).
The mean creativity_ai was 58.04 (SD = 26.54).
The mean creativity_human was 58.86 (SD = 23.51).
\begin{table}
\centering
\caption{Distribution of Gender}
\label{dist_gender}
\begin{tabular}{ll}
\toprule
{} & Counts (\% of total) \\
gender               &                     \\
\midrule
woman                &           296 (36\%) \\
man                  &           260 (32\%) \\
Missing              &           221 (27\%) \\
non-binary           &             22 (3\%) \\
prefer\_not\_disclose